In [1]:
import pandas as pd
import keras

from helpers import data, utils, generators

In [2]:
NUM_FOLDS = 10
NUM_EPOCHS = 3
INPUT_SHAPE = (192, 192, 1)
GPU_BATCH_SIZE = 16
DATA_BATCH_SIZE = 16

In [7]:
fold_dfs = data.get_fold_dfs(
    fold_definition_csv='../data/DATASET_CSVS/deepship_5k_seg_3s.csv',
    new_path_to_root='../data/deepship_baseline_unnorm_mat',
    ext='mat',
    unix=True,
    label_encodings={0:'Tanker', 1:'Cargo', 2:'Tug', 3:'Passengership'},
    n_folds=NUM_FOLDS
)
print('Successfully loaded folds.\n')

Successfully loaded folds.



In [8]:
train_df = pd.concat(fold_dfs[2:])
val_df = fold_dfs[1]
test_df = fold_dfs[0]

train_gen = generators.DeepShipGenerator(
    train_df,
    ext='mat',
    mat_var_name='Ptrans',
    batch_size=DATA_BATCH_SIZE
)
val_gen = generators.DeepShipGenerator(
    val_df,
    ext='mat',
    mat_var_name='Ptrans',
    batch_size=DATA_BATCH_SIZE
)
test_gen = generators.DeepShipGenerator(
    test_df,
    ext='mat',
    mat_var_name='Ptrans',
    batch_size=DATA_BATCH_SIZE
)

In [9]:
from models import cnn_lstm

cnn_lstm_model = cnn_lstm.get_cnn_lstm(
    input_shape=INPUT_SHAPE,
    input_name='spec',
    num_classes=4,
    verbose=True
)

cnn_lstm_model.compile(
    optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-5),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

Model: "cnn_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spec (InputLayer)           [(None, 192, 192, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 128)     3328      
                                                                 
 batch_normalization (Batch  (None, 192, 192, 128)     512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 192, 192, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 96, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 128)       409

In [10]:
keras.backend.clear_session()

history = cnn_lstm_model.fit(
    x=train_gen,
    validation_data=val_gen,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=2
)

Epoch 1/3


2024-11-20 01:57:27.130035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-20 01:57:28.589762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-20 01:57:29.442801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-20 02:08:43.271445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-20 02:08:43.452858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2716/2716 - 701s - loss: 0.8063 - acc: 0.6733 - val_loss: 0.8330 - val_acc: 0.6511 - 701s/epoch - 258ms/step
Epoch 2/3
2716/2716 - 698s - loss: 0.5118 - acc: 0.8086 - val_loss: 0.7495 - val_acc: 0.7021 - 698s/epoch - 257ms/step
Epoch 3/3
2716/2716 - 695s - loss: 0.3485 - acc: 0.8778 - val_loss: 0.7961 - val_acc: 0.6904 - 695s/epoch - 256ms/step


In [11]:
evals = cnn_lstm_model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

283/283 - 21s - loss: 0.8861 - acc: 0.6773 - 21s/epoch - 73ms/step
